# AirBNB Price Analyis

1. Webscrape AirBNB page to find absolute location of stay
2. Convert absolute location to address
4. Find estimated rent of the AirBNB
5. Infer actual cost of living in the AirBNB unit with expenses
6. Compare AirBNB price with estimated price, analyze the AirBNB premium

### Imports

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import requests
import json
import time

### Step 1: Webscraping AirBNB Rental

In [3]:
def getNumbers(w):
    nums = ""
    for char in w:
        if (char.isnumeric()):
            nums += char
    return int(nums)

In [27]:
options = Options()
options.add_argument("-headless")
driver = webdriver.Firefox(options=options)

In [20]:
listing_url = "https://www.airbnb.com/rooms/39446397"
driver.get(listing_url)
price, info, data = None, None, None
while (not price or not info or not data):
    try:
        time.sleep(1)
        data = driver.find_element("id", "data-deferred-state").get_attribute("innerHTML")
        price = int(getNumbers(driver.find_elements("class name", "_tyxjp1")[1].text.strip()))
        info = [getNumbers(_) for _ in driver.find_element("class name","lgx66tx").text.split("·")]
    except:
        newHeight = driver.execute_script("return document.documentElement.scrollHeight") + 50
        driver.execute_script("window.scroll(0, " + str(newHeight) + ");")
numGuests = info[0]
numBr = info[1]
numBed = info[2]
numBath =  info[3]
lastChars,lng,lat = "","",""
i = 0

# Basically just advanced String.find()
for char in data:
    if (len(lastChars) == 6):
        lastChars = lastChars[1:6] + char
    else:
        lastChars = lastChars + char
    if(lastChars == "\"lat\":"):
        lat = data[i+1:i+12]
    elif(lastChars == "\"lng\":"):
        lng = data[i+1:i+12]
    i+=1
while(not lat[len(lat)-1].isnumeric() or not lng[len(lng)-1].isnumeric()):
    if(lat[len(lat)-1].isnumeric()):
        lng = lng[:len(lng)-1]
    else:
        lat = lat[:len(lat)-1]
print("$" + str(price), "per night")
print(numBr,("Bedroom," if numBr == 1 else "Bedrooms,"),numBath,("Bathroom" if numBath == 1 else "Bathrooms"))
print(lat,lng,sep = ", ")

$117 per night
1 Bedroom, 1 Bathroom
38.97049, -77.00785


### Step 2. Getting AirBNB Address

In [21]:
api_token = "AAPK3a1b443d322b4817bcf976c67026e67aY3eBBi031akgsnQKM7K7YG2EfSTflxsLFa-iuS81KbBhqpVKs8PnIpZiiKTTmuRd"
loc_url = "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/reverseGeocode?f=pjson&location=" + lng + "," + lat + "&token=" + api_token 
response = requests.get(loc_url)
address_json = response.json()
address = address_json["address"]["Address"] + ", " + address_json["address"]["City"] + ", " + address_json["address"]["Region"] + ", " + address_json["address"]["Postal"]
print(address)

6617 Eastern Ave, Takoma Park, Maryland, 20912


### Step 3. Getting Associated Estimated Rent

In [22]:
sep_address = ""
for c in address:
    if(c == " "):
        sep_address += "%"
    elif(c.isalnum()):
        sep_address += c
headers = {"accept": "application/json"}
headers = {
    "accept": "application/json",
    "X-Api-Key": "949de555f3764976819af8d1e1dd6dbc"
}
property_url = "https://api.rentcast.io/v1/properties?address=" + sep_address
response = requests.get(property_url, headers=headers)

In [23]:
propertyType = "Apartment"
try:
    propertyType = response.json()["propertyType"]
except:
    pass
sep_property_type = ""
for c in propertyType:
    if(c == " "):
        sep_property_type += "%"
    elif(c.isalnum()):
        sep_property_type += c
print(sep_property_type)

Apartment


In [28]:
driver.get("https://app.rentcast.io/app")
elems = {"Text":[], "Element":[]}
for elem in driver.find_elements("class name", "dropdown-item"):
    elems[elem.get_attribute("textContent")] = elem
driver.find_element("class name", "form-control").send_keys(address)
driver.find_elements("class name", "dropdown-toggle")[3].click()
elems[" " + propertyType + " "].click()
driver.find_elements("class name", "dropdown-toggle")[4].click()
elems[" " + (str(numBr) if numBr < 6 else "6+") + (" Bed" if numBr == 1 else " Beds") + " "].click()
driver.find_elements("class name", "dropdown-toggle")[5].click()
elems[" " + (str(numBath) if numBath < 4 else "4+") + (" Bath" if numBath == 1 else " Baths") + " "].click()
driver.find_element("class name", "btn-primary").click()
rentEst = getNumbers(driver.find_element("class name", "display-3").get_attribute("textContent"))
print(rentEst)

2070


### 4. Infer actual cost of living in the AirBNB unit


### 5. Infer actual cost of living in the AirBNB unit


In [30]:
dayRentEst = round((rentEst / (365.25/12)),2)
premiumPerc = round(price/dayRentEst * 100)
print(premiumPerc)

172
